In [2]:
#imports 
import pandas as pd
import numpy as np

df = pd.read_excel('dnd-dataframe.xlsx', sheet_name=0, usecols=['name', 'weapons','herotype','spells'])
df.head(30)

,name,herotype,weapons,spells
0,bam,Bard,"Dagger, sling, club","Transmutation, Enchantment"
1,niem,Sorcerer,"light crossbow, battleaxe",Necromancy
2,aem,Paladin,Greataxe,"Abjuration, Conjuration"
3,yaeks,Rogue,"club, battleaxe","Conjuration, Evocation, Transmutation"
4,jeeks,Druid,"Dagger, Greataxe","Evocation, Transmutation, Necromancy"
5,aez,Sorcerer,"club, light crossbow","Evocation, Enchantment"
6,sax,Bard,"light crossbow, battleaxe, Dagger, sling, club",Necromancy
7,eert,Bard,Greataxe,"Transmutation, Necromancy"
8,wuc,Sorcerer,"light crossbow, battleaxe",Necromancy
9,yac,Sorcerer,Greataxe,"Conjuration, Evocation, Transmutation"


In [3]:


dummies1 = df['weapons'].str.get_dummies(sep=',')
dummies2 = df['spells'].str.get_dummies(sep=',')
dummies3 = df['herotype'].str.get_dummies(sep=',')

genre_data = pd.concat([df, dummies1,dummies2, dummies3], axis=1)

userInput = [
            {'name':'Irv', 'rating':1}   #Their is no rating system is being used thus by default rating is set to 1
         ] 
inputname = pd.DataFrame(userInput)

inputId = df[df['name'].isin(inputname['name'].tolist())]

#Then merging it so we can get the name. It's implicitly mergispellsng it by name.
inputname = pd.merge(inputId, inputname)

#Dropping information we won't use from the input dataframe
inputname = inputname.drop('weapons',1).drop('spells',1).drop('herotype',1)

#Filtering out the names from the input
username = genre_data[genre_data['name'].isin(inputname['name'].tolist())]

#Resetting the index to avoid future issues
username = username.reset_index(drop=True)

#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = username.drop('name',1).drop('weapons',1).drop('spells',1).drop('herotype',1)

#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputname['rating'])

genreTable = genre_data.copy()
genreTable = genreTable.drop('name',1).drop('weapons',1).drop('spells',1).drop('herotype',1)


In [4]:

#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())

#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)

#df.loc[df.index.isin(recommendationTable_df.head(3).keys())] #adjust the value of 3 here

df.loc[recommendationTable_df.head(6).index, :]

,name,herotype,weapons,spells
28,Irv,Sorcerer,light crossbow,Conjuration
9,yac,Sorcerer,Greataxe,"Conjuration, Evocation, Transmutation"
18,Traubon Durthane,Sorcerer,light crossbow,"Evocation, Transmutation, Necromancy"
8,wuc,Sorcerer,"light crossbow, battleaxe",Necromancy
1,niem,Sorcerer,"light crossbow, battleaxe",Necromancy
23,Zed Ryley,Sorcerer,sling,Evocation


In [5]:
recommendationTable_df.head(6)

28    1.000000
9     0.666667
18    0.666667
8     0.666667
1     0.666667
23    0.333333
dtype: float64